In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths

import os

import easydict

import torch
import torch.utils.data
from opts import opts
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory

In [2]:
import pixiedust

Pixiedust database opened successfully


In [3]:
opt = easydict.EasyDict({"K": 100,
"aggr_weight": 0.0,
"agnostic_ex": False,
"arch": "dla_34",
"aug_ddd": 0.5,
"aug_rot": 0,
"batch_size": 16,
"cat_spec_wh": False,
"center_thresh": 0.1,
"chunk_sizes": [15],
"data_dir": "/mnt/nas/data",
"dataset": "mobiltech",
"debug": 0,
"debug_dir": "/mnt/nas/centernet/exp/ctdet/mobiltech/mobiltech_dla/debug",
"debugger_theme": "white",
"demo": "/root/practice/CenterNet_objdet2D/images/mobiltech_test",
"dense_hp": False,
"dense_wh": False,
"dep_weight": 1,
"dim_weight": 1,
"down_ratio": 4,
"eval_oracle_dep": False,
"eval_oracle_hm": False,
"eval_oracle_hmhp": False,
"eval_oracle_hp_offset": False,
"eval_oracle_kps": False,
"eval_oracle_offset": False,
"eval_oracle_wh": False,
"exp_dir": "/mnt/nas/centernet/exp/ctdet",
"exp_id": "mobiltech_dla",
"fix_res": True,
"flip": 1.5,
"flip_test": True,
"gpus": [0],
"gpus_str": 0,
"head_conv": 256,
"heads": {'hm': 66, 'wh': 2, 'reg': 2},
"hide_data_time": False,
"hm_hp": True,
"hm_hp_weight": 1,
"hm_weight": 1,
"hp_weight": 1,
"input_h": 512,
"input_res": 512,
"input_w": 512,
"keep_res": False,
"kitti_split": "3dop",
"load_model": "/mnt/nas/centernet/exp/ctdet/mobiltech/mobiltech_dla/model_230.pth",
"lr": 0.0005,
"lr_step": [180, 210],
"master_batch_size": 15,
"mean": [[[0.40789655, 0.44719303, 0.47026116]]],
"metric": "loss",
"mse_loss": False,
"nms": False,
"no_color_aug": False,
"norm_wh": False,
"not_cuda_benchmark": False,
"not_hm_hp": False,
"not_prefetch_test": False,
"not_rand_crop": False,
"not_reg_bbox": False,
"not_reg_hp_offset": False,
"not_reg_offset": False,
"num_classes": 66,
"num_epochs": 230,
"num_iters": -1,
"num_stacks": 1,
"num_workers": 4,
"off_weight": 1,
"output_h": 128,
"output_res": 128,
"output_w": 128,
"pad": 31,
"peak_thresh": 0.2,
"print_iter": 0,
"rect_mask": False,
"reg_bbox": True,
"reg_hp_offset": True,
"reg_loss": "l1",
"reg_offset": True,
"resume": False,
"root_dir": "/mnt/nas",
"rot_weight": 1,
"rotate": 0,
"save_all": True,
"save_dir": "/mnt/nas/centernet/exp/ctdet/mobiltech/mobiltech_dla",
"scale": 0.4,
"scores_thresh": 0.1,
"seed": 317,
"shift": 0.1,
"std": [[[0.2886383, 0.27408165, 0.27809834]]],
"task": "ctdet",
"test": False,
"test_scales": [1.0],
"trainval": False,
"val_intervals": 5,
"vis_thresh": 0.3,
"wh_weight": 0.1})

In [4]:
torch.manual_seed(opt.seed)
torch.backends.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test
Dataset = get_dataset(opt.dataset, opt.task)
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
print(opt)

heads {'hm': 66, 'wh': 2, 'reg': 2}
{'K': 100, 'aggr_weight': 0.0, 'agnostic_ex': False, 'arch': 'dla_34', 'aug_ddd': 0.5, 'aug_rot': 0, 'batch_size': 16, 'cat_spec_wh': False, 'center_thresh': 0.1, 'chunk_sizes': [15], 'data_dir': '/mnt/nas/data', 'dataset': 'mobiltech', 'debug': 0, 'debug_dir': '/mnt/nas/centernet/exp/ctdet/mobiltech/mobiltech_dla/debug', 'debugger_theme': 'white', 'demo': '/root/practice/CenterNet_objdet2D/images/mobiltech_test', 'dense_hp': False, 'dense_wh': False, 'dep_weight': 1, 'dim_weight': 1, 'down_ratio': 4, 'eval_oracle_dep': False, 'eval_oracle_hm': False, 'eval_oracle_hmhp': False, 'eval_oracle_hp_offset': False, 'eval_oracle_kps': False, 'eval_oracle_offset': False, 'eval_oracle_wh': False, 'exp_dir': '/mnt/nas/centernet/exp/ctdet', 'exp_id': 'mobiltech_dla', 'fix_res': True, 'flip': 1.5, 'flip_test': True, 'gpus': [0], 'gpus_str': 0, 'head_conv': 256, 'heads': {'hm': 66, 'wh': 2, 'reg': 2}, 'hide_data_time': False, 'hm_hp': True, 'hm_hp_weight': 1, 'hm

In [5]:
model = create_model(opt.arch, opt.heads, opt.head_conv).cuda()
#torch.save(model, 'model.pth')

In [6]:
from torch.autograd import Variable
import torch.onnx

optimizer = torch.optim.Adam(model.parameters(), opt.lr)
model= load_model(model, opt.load_model).cuda()
dummy_input = Variable(torch.randn(16,3,512,512)).cuda()
model.train(False)
#traced_model = torch.jit.trace(model, dummy_input)

loaded /mnt/nas/centernet/exp/ctdet/mobiltech/mobiltech_dla/model_230.pth, epoch 230


DLASeg(
  (base): DLA(
    (base_layer): Sequential(
      (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
    )
    (level0): Sequential(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
    )
    (level1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
    )
    (level2): Tree(
      (tree1): BasicBlock(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inpla

In [7]:
torch.onnx.export(model, dummy_input, "centernet.onnx", export_params=True)

In [1]:
import onnx

model = onnx.load("centernet.onnx")

onnx.checker.check_model(model)

onnx.helper.printable_graph(model.graph)

'graph torch-jit-export (\n  %0[FLOAT, 16x3x512x512]\n) initializers (\n  %1[FLOAT, 16x3x7x7]\n  %2[FLOAT, 16]\n  %3[FLOAT, 16]\n  %4[FLOAT, 16]\n  %5[FLOAT, 16]\n  %6[INT64, scalar]\n  %7[FLOAT, 16x16x3x3]\n  %8[FLOAT, 16]\n  %9[FLOAT, 16]\n  %10[FLOAT, 16]\n  %11[FLOAT, 16]\n  %12[INT64, scalar]\n  %13[FLOAT, 32x16x3x3]\n  %14[FLOAT, 32]\n  %15[FLOAT, 32]\n  %16[FLOAT, 32]\n  %17[FLOAT, 32]\n  %18[INT64, scalar]\n  %19[FLOAT, 64x32x3x3]\n  %20[FLOAT, 64]\n  %21[FLOAT, 64]\n  %22[FLOAT, 64]\n  %23[FLOAT, 64]\n  %24[INT64, scalar]\n  %25[FLOAT, 64x64x3x3]\n  %26[FLOAT, 64]\n  %27[FLOAT, 64]\n  %28[FLOAT, 64]\n  %29[FLOAT, 64]\n  %30[INT64, scalar]\n  %31[FLOAT, 64x64x3x3]\n  %32[FLOAT, 64]\n  %33[FLOAT, 64]\n  %34[FLOAT, 64]\n  %35[FLOAT, 64]\n  %36[INT64, scalar]\n  %37[FLOAT, 64x64x3x3]\n  %38[FLOAT, 64]\n  %39[FLOAT, 64]\n  %40[FLOAT, 64]\n  %41[FLOAT, 64]\n  %42[INT64, scalar]\n  %43[FLOAT, 64x128x1x1]\n  %44[FLOAT, 64]\n  %45[FLOAT, 64]\n  %46[FLOAT, 64]\n  %47[FLOAT, 64]\n  %48[I